In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn import svm

In [2]:

def read_images(file,image_size,color):

    images = []
    for img in os.listdir(file):

        image = cv2.imread(file+"/"+img,color)
     
        image = cv2.resize(image,image_size)
       

        images.append(image)

    return np.array(images)

def save_images(image_array,folder):
    dir = os.path.join(os.getcwd(),folder)

    if not os.path.exists(dir):
        os.mkdir(folder)
    
    image_path = os.path.join(folder)  
    for i in range(len(image_array)):
        image_name = folder +"_"+str(i+1)+".jpg"
        image_path = os.path.join(folder,image_name)  
        cv2.imwrite(image_path,image_array[i])
        

def applyCanny(image_array,l_Threshold,h_threshold):
    
    canny_Array = np.zeros((image_array.shape[0],image_array.shape[1],image_array.shape[2])) # 3D array
    for i in range(len(image_array)):
        gauss_img = cv2.GaussianBlur(image_array[i],(5,5),1)  # Gaussian Blur before edge detection
        canny_Array[i] = cv2.Canny(gauss_img,l_Threshold,h_threshold) # Canny Edge Detection
    return canny_Array




def crop_circle(image, circle):
    
    x, y, r = circle
    mask = np.zeros_like(image)  
    cv2.circle(mask, (x, y), r, (255, 255, 255), -1)
    cropped = image * mask
    cropped = cv2.resize(cropped,(128,128)) # 100,100
    return cropped

def calculateHOG(image_gray):
    cell_size = 8 
    block_size = 2 
    bin_num = 9

    x,y = image_gray.shape 

    image = image_gray.flatten()

    n_cells_x = int(x // cell_size) # number of cells in x direction
    n_cells_y = int(y // cell_size) # number of cells in y direction

    n_blocks_y = n_cells_y - block_size + 1 
    n_blocks_x = n_cells_x - block_size + 1

    x_filter = np.array([-1,0,1]) 
    y_filter = x_filter.T

    gx = (np.convolve(image,x_filter,mode="same")).reshape(x,y) # gradient in x direction
    gy = (np.convolve(image,y_filter,mode="same")).reshape(x,y) # gradient in y direction

   

    magnitude = np.sqrt(gx**2 + gy**2)
    orientation = np.arctan2(gy,gx) * (180/np.pi) # orientation in degrees

    orientation  = orientation % 180 # orientation in range 0 to 180

    bin_edges = np.linspace(0,180,bin_num + 1) # bin edges for histogram
    #bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2 

    hog_cells = np.zeros((n_cells_y,n_cells_x,bin_num)) 

    for i in range(n_cells_y): 
        for j in range(n_cells_x): 
            cell_mag = magnitude[i*cell_size : (i+1) * cell_size , j*cell_size : (j+1) * cell_size] # magnitude of cell
            cell_ori = orientation[i*cell_size : (i+1) * cell_size , j*cell_size : (j+1) * cell_size] # orientation of cell

            cell_bins = np.digitize(cell_ori,bin_edges)  # bin number of each pixel in cell

            for k in range(1,bin_num +1 ):
                hog_cells[i,j,k-1] = np.sum(cell_mag[cell_bins == k]) # sum of magnitude of pixels in each bin

    
    hog_blocks = np.zeros((n_blocks_y,n_blocks_x,block_size ** 2 *bin_num)) # hog features of each block
    for i in range(n_blocks_y):
        for j in range(n_blocks_x):
            block_hog = hog_cells[i:i+block_size,j:j+block_size,:] 
            block_hog = block_hog.ravel() # flatten the block hog features

            block_hog = block_hog / np.sqrt(np.sum(block_hog**2) + 1e-6) # normalizing the block hog features

            hog_blocks[i,j,:] = block_hog

    
    hog_features = hog_blocks.ravel()
    
    return hog_features


In [3]:
train_images = read_images("Train",(400,400),1)
test_images_R = read_images("TestR",(1500,1000),1)
test_images_V = read_images("TestV",(1500,1000),1)


train_images_gray = read_images("Train",(100,100),0) 
test_images_R_gray = read_images("TestR",(300,200),0)
test_images_V_gray = read_images("TestV",(300,200),0)

train_scale = 4
test_scale = 5



In [4]:
low_t = 50   # 50
high_t = 150  # 80
canny_train_images = applyCanny(train_images_gray,low_t,high_t)
low_t = 0
high_t = 15  
canny_test_images_R = applyCanny(test_images_R_gray,low_t,high_t)
canny_test_images_V = applyCanny(test_images_V_gray,low_t,high_t)

In [5]:


def hough_circle_transform(image, min_rad, max_rad, threshold, circle_num):
    y, x = np.nonzero(image)  # Pixel coordinates of edges

    # Accumulator array to store votes
    accumulator_arr = np.zeros((max_rad, image.shape[0], image.shape[1]))

    theta = np.arange(0, 2 * np.pi, np.pi / 180)
    sin = np.sin(theta)
    cos = np.cos(theta)

    for i in range(len(x)):
        for r in range(min_rad, max_rad):  # Since radius is unknown, loop is created for each radius value.
            a = (x[i] - r * cos)
            b = (y[i] - r * sin)
            a = a.astype(int)
            b = b.astype(int)

            valid = np.where((a >= 0) & (a < image.shape[1]) & (b >= 0) & (b < image.shape[0]))

            a = a[valid]
            b = b[valid]

            # Vote
            accumulator_arr[r, b, a] += 1

    circles = np.where(accumulator_arr > threshold)  # Find circles with votes greater than threshold

        # Non-maximum suppression
    sorted_indices = np.argsort(accumulator_arr[circles], axis=None)[::-1]
    sorted_circles = (circles[0][sorted_indices], circles[1][sorted_indices], circles[2][sorted_indices])

    selected_circles = []
    for i in range(len(sorted_indices)):
        r = sorted_circles[0][i]
        x = sorted_circles[1][i]
        y = sorted_circles[2][i]

        overlap = False
        for selected_circle in selected_circles:
            dist = np.sqrt((x - selected_circle[0]) ** 2 + (y - selected_circle[1]) ** 2)
            if dist < selected_circle[2] or dist < r:
                overlap = True
                break

        if not overlap:
            selected_circles.append((x, y, r))

        if len(selected_circles) >= circle_num:
            break

    x = [circle[0] for circle in selected_circles]
    y = [circle[1] for circle in selected_circles]
    radius = [circle[2] for circle in selected_circles]

    return list(zip(radius, x, y))


In [6]:
circled_imgs_train = []
circles_train = []
for i in range(len(train_images)):
    
    circles = hough_circle_transform(canny_train_images[i],35,50,20,circle_num=1)
    circles_train.append(circles)

    for r in circles:
        train_images[i] = cv2.circle(train_images[i],(r[2]*train_scale,r[1]*train_scale),r[0]*train_scale,(255,0,0),2)
    circled_imgs_train.append(train_images[i])
    print(f"{i+1}. image completed.")

1. image completed.
2. image completed.
3. image completed.
4. image completed.
5. image completed.
6. image completed.
7. image completed.
8. image completed.
9. image completed.
10. image completed.
11. image completed.
12. image completed.
13. image completed.
14. image completed.
15. image completed.
16. image completed.
17. image completed.
18. image completed.
19. image completed.
20. image completed.
21. image completed.
22. image completed.
23. image completed.
24. image completed.
25. image completed.
26. image completed.
27. image completed.
28. image completed.
29. image completed.
30. image completed.
31. image completed.
32. image completed.
33. image completed.
34. image completed.
35. image completed.
36. image completed.
37. image completed.
38. image completed.
39. image completed.
40. image completed.
41. image completed.
42. image completed.
43. image completed.
44. image completed.
45. image completed.
46. image completed.
47. image completed.
48. image completed.
4

In [7]:
circled_imgs_R = []
circles_R = []
for i in range(len(test_images_R)):
    
    circles = hough_circle_transform(canny_test_images_R[i],1,15,5,circle_num=20)
    circles_R.append(circles)
    for r in circles:
        test_images_R[i] = cv2.circle(test_images_R[i],(r[2]*test_scale,r[1]*test_scale),r[0]*test_scale,(255,0,0),2)
        
    circled_imgs_R.append(test_images_R[i])
    print(f"{i+1}. image completed.")


1. image completed.
2. image completed.
3. image completed.
4. image completed.


In [8]:
circled_imgs_V = []
circles_V = []
for i in range(len(test_images_V)):
    circles = hough_circle_transform(canny_test_images_V[i],1,15,5,circle_num=20)
    circles_V.append(circles)
    for r in circles:
        test_images_V[i] = cv2.circle(test_images_V[i],(r[2]*test_scale,r[1]*test_scale),r[0]*test_scale,(255,0,0),2)
        
    circled_imgs_V.append(test_images_V[i])
    print(f"{i+1}. image completed.")

1. image completed.
2. image completed.
3. image completed.
4. image completed.
5. image completed.
6. image completed.
7. image completed.
8. image completed.
9. image completed.
10. image completed.
11. image completed.
12. image completed.


In [9]:
save_images(circled_imgs_train,"Train_Hough")
save_images(circled_imgs_R,"TestR_Hough")
save_images(circled_imgs_V,"TestV_Hough")



In [10]:
labels = []

for img_name in os.listdir("Train"):
    image_list = img_name.split("_")
    label_Name = image_list[0] + "_" + image_list[1]
    labels.append(label_Name)

print(labels)

['10kr_obverse', '10kr_obverse', '10kr_obverse', '10kr_obverse', '10kr_obverse', '10kr_reverse', '10kr_reverse', '10kr_reverse', '10kr_reverse', '10kr_reverse', '1kr_obverse', '1kr_obverse', '1kr_obverse', '1kr_obverse', '1kr_reverse', '1kr_reverse', '1kr_reverse', '1kr_reverse', '1TL_obverse', '1TL_obverse', '1TL_obverse', '1TL_obverse', '1TL_obverse', '1TL_reverse', '1TL_reverse', '1TL_reverse', '1TL_reverse', '1TL_reverse', '25kr_obverse', '25kr_obverse', '25kr_obverse', '25kr_obverse', '25kr_obverse', '25kr_reverse', '25kr_reverse', '25kr_reverse', '25kr_reverse', '25kr_reverse', '50kr_obverse', '50kr_obverse', '50kr_obverse', '50kr_obverse', '50kr_obverse', '50kr_reverse', '50kr_reverse', '50kr_reverse', '50kr_reverse', '50kr_reverse', '5kr_obverse', '5kr_obverse', '5kr_obverse', '5kr_obverse', '5kr_reverse', '5kr_reverse', '5kr_reverse', '5kr_reverse']


In [11]:
train_x = []
train_y = []


for i in range(len(train_images_gray)): # For each train image (with class), tüm çemberlerin oluşturduğu hogları bul
        if(len(circles_train[i]) == 0):
                continue
        cropped_img = crop_circle(train_images_gray[i],circles_train[i][0])
        train_x.append(calculateHOG(cropped_img)) 
        train_y.append(labels[i])

train_x = np.array(train_x)
train_y = np.array(train_y)







In [12]:
classifier = svm.SVC()
classifier.fit(train_x,train_y)

SVC()

In [13]:

for i in range(len(test_images_V_gray)):
        if(len(circles_V[i]) == 0):
                continue
        for circle in circles_V[i]:
                cropped_img = crop_circle(test_images_V_gray[i],circle)
                hog = calculateHOG(cropped_img)
                prediction = classifier.predict([hog])
                
                test_images_V[i] = cv2.putText(test_images_V[i],prediction[0],(circle[2]*test_scale,circle[1]*test_scale),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,cv2.LINE_AA)


save_images(test_images_V,"TestV_HoG")




In [14]:
for i in range(len(test_images_R_gray)):
        if(len(circles_R[i]) == 0):
                continue
        for circle in circles_R[i]:
                cropped_img = crop_circle(test_images_R_gray[i],circle)
                hog = calculateHOG(cropped_img)
                prediction = classifier.predict([hog])
                
                test_images_R[i] = cv2.putText(test_images_R[i],prediction[0],(circle[2]*test_scale,circle[1]*test_scale),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,cv2.LINE_AA)


save_images(test_images_R,"TestR_HoG")
